# Query GEE and download ArcticDEM Strips

In [ ]:
import ee
import geopandas as gpd
import geedim as gd
import matplotlib.pyplot as plt
import os
import xarray as xr
import numpy as np
import datetime
import glob
import rasterio as rio

In [ ]:
# -----Define output directory for DEMs
out_path = '/Users/raineyaberle/Research/PhD/Hubbard/surface_elevations/ArcticDEM/'

# -----Load IFSAR DEM for reference
ifsar_fn = '/Users/raineyaberle/Research/PhD/Hubbard/surface_elevations/ifsar_hubbardDEM.tif'
ifsar = xr.open_dataset(ifsar_fn)
ifsar = ifsar.rio.reproject('EPSG:4326')

# -----Load glacier boundaries
aoi_fn = '/Users/raineyaberle/Research/PhD/Hubbard/glacier_boundaries/Hubbard_boundaries.shp'
aoi = gpd.read_file(aoi_fn)
# reproject to Alaska Albers projection
# aoi_utm = aoi.to_crs('EPSG:3338')
# # buffer by 1 km
# aoi_utm_buffer = aoi_utm.buffer(1e3)
# # reproject back to WGS84
# aoi_buffer = aoi_utm_buffer.to_crs('EPSG:4326')
# # construct bounding box
# bbox = [[aoi_buffer.geometry.bounds.minx[0], aoi_buffer.geometry.bounds.miny[0]],
#          [aoi_buffer.geometry.bounds.maxx[0], aoi_buffer.geometry.bounds.miny[0]],
#          [aoi_buffer.geometry.bounds.maxx[0], aoi_buffer.geometry.bounds.maxy[0]],
#          [aoi_buffer.geometry.bounds.minx[0], aoi_buffer.geometry.bounds.maxy[0]],
#          [aoi_buffer.geometry.bounds.minx[0], aoi_buffer.geometry.bounds.miny[0]]]

# use a smaller bounding box to cover the terminus
minx, maxx = np.min(ifsar.x.data), np.max(ifsar.x.data)
miny, maxy = np.min(ifsar.y.data), np.max(ifsar.y.data)
bbox = [[minx, miny], [maxx, miny], [maxx, maxy], [minx, maxy], [minx, miny]]

# plot
fig, ax = plt.subplots()
aoi.plot(ax=ax, facecolor='None', edgecolor='b')
ax.imshow(ifsar.band_data.data[0], cmap='terrain', clim=(0,2.5e3),
          extent=(np.min(ifsar.x.data), np.max(ifsar.x.data), 
                  np.min(ifsar.y.data), np.max(ifsar.y.data)))
ax.plot([x[0] for x in bbox], [x[1] for x in bbox], '-k')
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.show()
# Reformat for querying GEE
region = {'type': 'Polygon',
          'coordinates': [bbox]
          }

## Query geedim for ArcticDEM strips

In [ ]:
# Initialize GEE
ee.Initialize()

In [ ]:
# Define function for creating a list of image IDs to mosaic
def image_mosaic_ids(im_col_gd):
    
    # Grab image properties, IDs, and datetimes from image collection
    properties = im_col_gd.properties
    ims = dict(properties).keys()
    im_ids = [properties[im]['system:id'] for im in ims]
    # return if no images found
    if len(im_ids) < 1:
        return 'N/A', 'N/A'
    im_dts = [datetime.datetime.utcfromtimestamp(properties[im]['system:time_start'] / 1000) for im in ims]
    
    # Initialize list of image ids and datetimes to mosaic
    im_mosaic_ids_list_full, im_mosaic_dts_list_full = [], []
    
    # Grab all unique hours in image datetimes
    hours = np.array(im_dts, dtype='datetime64[h]')
    unique_hours = sorted(set(hours))
    
    # Create list of IDs for each unique hour
    im_ids_list, im_dts_list = [], []
    for unique_hour in unique_hours:
        i = list(np.ravel(np.argwhere(hours == unique_hour)))
        im_ids_list_hour = [im_ids[ii] for ii in i]
        im_ids_list = im_ids_list + [im_ids_list_hour]
        im_dts_list_hour = [im_dts[ii] for ii in i]
        im_dts_list = im_dts_list + [im_dts_list_hour]
    im_mosaic_ids_list_full = im_mosaic_ids_list_full + im_ids_list
    im_mosaic_dts_list_full = im_mosaic_dts_list_full + im_dts_list
    
    return im_mosaic_ids_list_full, im_mosaic_dts_list_full

In [ ]:
# Create image collection, filter by date and region
im_collection = gd.MaskedCollection.from_name("UMN/PGC/ArcticDEM/V3/2m")
im_collection_filt = im_collection.search('2000-01-01', '2024-02-15', region)
print(im_collection_filt.properties_table)
# Create a list of image IDs to mosaic
im_ids_list, im_dts_list = image_mosaic_ids(im_collection_filt)
im_ids_list

In [ ]:
# # Get image thumbnails
# import PIL
# import requests
# import io

# def get_image_thumbnail(im_ee):
#     visualization = {'bands': 'elevation', 'min': 0, 'max': 2.5e3, 'region': region, 'dimensions': 768}
   
    
#     image_url = im_ee.clip(region).getThumbURL(visualization)
#     # Fetch the image and convert it to a PIL Image object
#     response = requests.get(image_url)
#     image_bytes = io.BytesIO(response.content)
#     image = PIL.Image.open(image_bytes)
#     return image

# # Reproject the Earth Engine image to UTM projection
# utm_epsg = 'EPSG:3338'
# im_collection_filt_utm = im_collection_filt.ee_collection.map(lambda img: img.reproject(crs=f'EPSG:{utm_epsg}', scale=30))
# im_ids = list(im_collection_filt.properties)
# images = [get_image_thumbnail(ee.Image(im_id)) for im_id in im_ids]
# images

In [ ]:
# for image in images:
#     plt.imshow(image, cmap='terrain', extent=(minx, maxx, miny, maxy))
#     plt.show()

In [ ]:
# iterate over list of image IDs
for i in tqdm(range(0, len(im_ids_list[0:1]))):

    # subset image IDs and image datetimes
    im_ids, im_dt = im_ids_list[i], im_dts_list[i]

    # define filename
    if len(im_ids) > 1:
        im_fn =  'ArcticDEM_Hubbard_' + str(im_dt[0]).replace('-', '')[0:8] + '_MOSAIC.tif'
    else:
        im_fn = 'ArcticDEM_Hubbard_' + str(im_dt[0]).replace('-', '')[0:8] + '.tif'
    # check file does not already exist in directory, download
    if not os.path.exists(os.path.join(out_path, im_fn)):
        # create list of MaskedImages from IDs
        im_gd_list = [gd.MaskedImage.from_id(im_id) for im_id in im_ids]
        # combine into new MaskedCollection
        im_collection = gd.MaskedCollection.from_list(im_gd_list)
        # create image composite
        im_composite = im_collection.composite(region=region)
        # download to file
        im_composite.download(os.path.join(out_path, im_fn),
                              region=region,
                              scale=2,
                              crs='EPSG:4326',
                              dtype='int16')
    else:
        print('Image already exists in directory, skipping...')
        continue

In [ ]:
# -----Plot images
# reproject AOI to Alaska Albers projection
aoi_utm = aoi.to_crs('EPSG:3338')
dem_fns = sorted(glob.glob(os.path.join(out_path, '*.tif')))
for dem_fn in dem_fns:
    if 'MOSAIC' in dem_fn:
        date = dem_fn[-19:-15] + '-' + dem_fn[-15:-13] + '-' + dem_fn[-13:11]
    else:
        date = dem_fn[-12:-8] + '-' + dem_fn[-8:-6] + '-' + dem_fn[-6:-4]
    dem = xr.open_dataset(dem_fn)
    dem = dem.rio.reproject('EPSG:3338')
    fig, ax = plt.subplots(1, 1, figsize=(6,6))
    dem_im = ax.imshow(elevation, extent=(np.min(dem.x.data), np.max(dem.x.data), 
                                          np.min(dem.y.data), np.max(dem.y.data)), 
                       cmap='terrain', clim=(0,2.5e3))
    aoi_utm.plot(ax=ax, facecolor='None', edgecolor='k')
    fig.colorbar(dem_im, ax=ax, shrink=0.5, 
                 label='Elevation w.r.t. ellipsoid [m]')
    ax.set_title(date)
    ax.set_xlim(7.9e5, 8.15e5)
    ax.set_ylim(1.194e6, 1.22e6)
    ax.grid()
    plt.show()